In [11]:
import os
import gensim
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
from datasets import list_datasets, load_dataset

In [13]:
question = None
written = False

with open("prepared_answers.txt", "w") as fout:  
    with open("Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

AssertionError: 

In [14]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [15]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

/tmp/ipykernel_105198/1675038189.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [16]:
# Классифицируем текст с помощью FastText

sentences = [i for i in sentences if len(i) > 2]
modelFT = FastText(sentences=sentences, vector_size=100, min_count=1, window=5)
modelFT.save("ft_model")

In [17]:
# Создаем Индексы для вопросов-ответов
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)


modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

        if counter > 500000:
            break

ft_index.build(10)
ft_index.save('speaker.ann')

/tmp/ipykernel_105198/1814660221.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


0it [00:00, ?it/s]

True

In [18]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann') 

True

In [19]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[2111, 5846]

In [20]:
#. Создадим модель данных

data = load_dataset('Den4ikAI/russian_code_qa')
python_data = pd.DataFrame(data['train'])

python_data['question'] = python_data['question'].apply(lambda x: preprocess_txt(str(x)))

python_data.head()

Found cached dataset json (/home/kmv/.cache/huggingface/datasets/Den4ikAI___json/Den4ikAI--russian_code_qa-989d04dd8d675d6a/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

,question,code
0,"[корреляция, переменный, временной, ряд, python]",ts1.corr(ts2)
1,"[перенаправить, стандартный, вывод, регистрато...",logging.basicConfig(level=logging.DEBUG)
2,"[эффективный, способ, преобразование, строка, ...",return [int(x) for x in '8743-12083-15'.split(...
3,"[панда, сгруппировать, поле, объединить, значе...","df = pd.DataFrame(data, columns=['id', 'A', 'B..."
4,"[получить, определённый, число, массив, python]",[0] * 3


In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [22]:
idxs = set(np.random.randint(0, len(index_map), len(python_data)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in python_data['question'].values]

In [23]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=13)

In [25]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

/home/kmv/projects/data_science/dsvenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9931290726115125

In [27]:
assert True
import pickle

with open("index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [29]:
np.mean(tfidf_vect.idf_)

12.250986538611183

In [30]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'dict': 9.304532241006095,
 'python': 7.0989568844524555,
 'использовать': 10.330385175391777,
 'unicode': 11.888529793438327,
 'вместо': 10.635766824942959,
 'байтовый': 12.581676973998272,
 'строка': 12.581676973998272,
 'словарь': 12.581676973998272,
 'генерация': 12.581676973998272,
 'подмножество': 12.581676973998272,
 'переставить': 12.581676973998272,
 'список': 11.888529793438327,
 'слово': 12.581676973998272,
 'скрыть': 12.581676973998272,
 'метка': 12.581676973998272,
 'тиковый': 12.581676973998272,
 'оставить': 11.888529793438327,
 'галочка': 12.581676973998272,
 'место': 12.581676973998272,
 'право': 12.581676973998272,
 'сердце': 12.581676973998272,
 'легко': 12.581676973998272,
 'преобразовать': 12.581676973998272,
 'шестнадцатеричный': 12.581676973998272,
 'ascii': 12.581676973998272,
 'необработанный': 12.581676973998272,
 'внутренний': 12.581676973998272,
 'двоичный': 12.581676973998272,
 'graphviz': 12.581676973998272,
 'рисование': 12.581676973998272,
 'максимальный

In [31]:
# Создаем Индексы для продуктовых данных

ft_index_python = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_python = {}
counter = 0

for i in tqdm_notebook(range(len(python_data))):
    n_ft = 0
    index_map_python[counter] = (python_data.loc[i, "question"], python_data.loc[i, "code"])
    vector_ft = np.zeros(100)
    for word in python_data.loc[i, "question"]:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_python.add_item(counter, vector_ft)
    counter += 1

ft_index_python.build(10)
ft_index_python.save('python.ann')

/tmp/ipykernel_105198/2633034149.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(python_data))):


  0%|          | 0/143253 [00:00<?, ?it/s]

True

In [32]:
assert True
import pickle

with open("index_python.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [33]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [34]:
ft_index_python.get_nns_by_vector(np.zeros(100), 1, include_distances=True)

([909], [1.4142135381698608])

In [ ]:
updater = Updater("5839074559:AAFmKifiny0hp7c9n4n1dKApkk6g4JgZYak", use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Привет, давай пообщаемся?')
def textMessage3(update, context):
    response = 'Получил Ваше сообщение: ' + str(update.message.text)
    context.bot.send_message(chat_id=update.message.chat_id, text=response)



def startCommand2(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Добрый день')

def textMessage(update, context):

    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)

    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_python_val = ft_index_python.get_nns_by_vector(vect_ft, 1)
        for item in ft_index_python_val:
            question, code = index_map_python[item]
            context.bot.send_message(chat_id=update.message.chat_id, text=code)
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.2:
        print(distances[0])
        context.bot.send_message(chat_id=update.message.chat_id, text="Моя твоя не понимать")
        return
    context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])

start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()